In [2]:
import json
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import random # For randomized search
import sys # For min_delta

# -------------------- Utility Functions --------------------
def mean_absolute_percentage_error(y_true, y_pred):
    """
    Calculates MAPE, preventing division by zero.
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    # Add a small epsilon to prevent division by zero for y_true values that are zero
    epsilon = 1e-10
    return np.mean(np.abs((y_true - y_pred) / (y_true + epsilon))) * 100

# -------------------- Device Setup --------------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# -------------------- Step 1: 데이터 로드 및 전처리 (JSON 파일용) --------------------
# 'review.json' 파일을 올바르게 로드합니다.
try:
    # JSON 파일이 한 줄에 하나의 JSON 객체로 되어 있는 경우 (JSONL 형식)
    df = pd.read_json('review.json', lines=True)
    print("✅ JSON 파일 로드 성공.")
except FileNotFoundError:
    print("❌ Error: 'review.json' 파일이 존재하지 않습니다.")
    sys.exit()
except ValueError as e:
    # JSON 파일이 단일 JSON 배열인 경우
    print(f"JSONL 형식이 아닌 것 같습니다. 일반 JSON 파일로 다시 시도합니다. (오류: {e})")
    try:
        df = pd.read_json('review.json')
        print("✅ 일반 JSON 파일 로드 성공.")
    except Exception as e:
        print(f"❌ Error: 'review.json' 파일을 읽는 데 실패했습니다. 파일 형식을 확인해주세요. (오류: {e})")
        sys.exit()

# 필요한 컬럼만 선택하고, 결측치를 제거합니다.
df = df[['user_id', 'business_id', 'stars', 'text']].dropna()

# -------------------- SBERT 모델 로딩 (한 번만) --------------------
print("SBERT 모델 로딩 중...")
sbert_model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
print("SBERT 모델 로딩 완료.")

# -------------------- Dataset 클래스 정의 --------------------
class UCAMDataset(Dataset):
    def __init__(self, users, items, ratings, contexts):
        self.users = users
        self.items = items
        self.ratings = ratings
        self.contexts = contexts

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        return (
            torch.tensor(self.users[idx], dtype=torch.long),
            torch.tensor(self.items[idx], dtype=torch.long),
            torch.tensor(self.contexts[idx], dtype=torch.float32),
            torch.tensor(self.ratings[idx], dtype=torch.float32)
        )

# -------------------- 모델 정의 --------------------
class UCAM(nn.Module):
    def __init__(self, num_users, num_items, context_dim=384, embed_dim=64, hidden_dims=[128, 64]):
        super().__init__()
        # '알 수 없음' 인덱스를 위해 +1
        self.user_embed = nn.Embedding(num_users + 1, embed_dim)
        self.item_embed = nn.Embedding(num_items + 1, embed_dim)

        # MLP 레이어 동적으로 생성
        layers = []
        input_dim = embed_dim * 2 + context_dim
        for h_dim in hidden_dims:
            layers.append(nn.Linear(input_dim, h_dim))
            layers.append(nn.ReLU())
            input_dim = h_dim
        
        layers.append(nn.Linear(input_dim, 1)) # 최종 출력 레이어
        self.fc_layers = nn.Sequential(*layers)

    def forward(self, user_ids, item_ids, context_vecs):
        u = self.user_embed(user_ids)
        i = self.item_embed(item_ids)
        x = torch.cat([u, i, context_vecs], dim=-1)
        return self.fc_layers(x).squeeze()

# -------------------- 평가 지표 함수 --------------------
def evaluate_model(model, data_loader, device):
    model.eval()
    preds, targets = [], []

    with torch.no_grad():
        for users, items, contexts, ratings in data_loader:
            users = users.to(device)
            items = items.to(device)
            contexts = contexts.to(device)
            ratings = ratings.to(device)

            output = model(users, items, contexts)
            preds.extend(output.cpu().numpy())
            targets.extend(ratings.cpu().numpy())

    preds = np.array(preds)
    targets = np.array(targets)

    mae = mean_absolute_error(targets, preds)
    mse = mean_squared_error(targets, preds)
    rmse = np.sqrt(mse)
    mape = mean_absolute_percentage_error(targets, preds)

    return mae, mse, rmse, mape

# -------------------- Hyperparameter Search Setup --------------------
param_grid = {
    'embed_dim': [32, 64, 128], # Embedding dimension for users and items
    'hidden_dims': [[64, 32], [128, 64], [256, 128]], # Hidden layers for MLP
    'learning_rate': [0.0005, 0.001, 0.002],
    'batch_size': [128, 256, 512],
    'patience': [5, 7, 10] # Early stopping patience
}

num_trials = 10 # Number of random combinations to try
best_params = None
best_rmse = float('inf')
results_log = [] # To store results of each trial
min_delta = 1e-4 # Minimum change to be considered an improvement

# Fixed random state for data splitting. This ensures that
# all hyperparameter trials use the exact same train/val/test splits,
# making the comparison of hyperparameters fair.
DATA_SPLIT_RANDOM_STATE = 42

print(f"\n--- Starting Hyperparameter Search with {num_trials} trials ---")
print(f"Data Split Random State: {DATA_SPLIT_RANDOM_STATE}")

# Perform data splitting once
train_val_df, test_df = train_test_split(df, test_size=0.2, random_state=DATA_SPLIT_RANDOM_STATE)

# 훈련 세트의 고유한 사용자/아이템 ID를 기반으로 인덱스를 생성합니다.
user2idx = {uid: i for i, uid in enumerate(train_val_df['user_id'].unique())}
item2idx = {iid: i for i, iid in enumerate(train_val_df['business_id'].unique())}

# 테스트 세트에만 존재하는 사용자/아이템을 위한 '알 수 없음' 인덱스를 추가합니다.
# 이 인덱스는 train_val_df에 없는 ID를 처리합니다.
unknown_user_idx = len(user2idx)
unknown_item_idx = len(item2idx)

# SBERT 문맥 벡터 생성 (전체 데이터에서 한 번만)
print("SBERT 문맥 벡터 생성 중 (전체 데이터)...")
# Note: For efficiency, we encode the full text column and then split
# This avoids re-encoding the same texts if they appear in different splits.
all_texts = df['text'].tolist()
all_context_vectors = sbert_model.encode(all_texts, show_progress_bar=True)
df['context_vectors'] = list(all_context_vectors) # Add as a new column

# Split dataframes with context vectors
train_val_df_with_contexts = df.loc[train_val_df.index]
test_df_with_contexts = df.loc[test_df.index]

# Map user/item IDs to indices for all splits
train_val_df_with_contexts['user_idx'] = train_val_df_with_contexts['user_id'].map(user2idx)
train_val_df_with_contexts['item_idx'] = train_val_df_with_contexts['business_id'].map(item2idx)
test_df_with_contexts['user_idx'] = test_df_with_contexts['user_id'].map(user2idx).fillna(unknown_user_idx).astype(int)
test_df_with_contexts['item_idx'] = test_df_with_contexts['business_id'].map(item2idx).fillna(unknown_item_idx).astype(int)

# Further split train_val into train and validation for early stopping
train_df, val_df = train_test_split(train_val_df_with_contexts, test_size=0.125, random_state=DATA_SPLIT_RANDOM_STATE) # 0.125 * 0.8 = 0.1 (10%)

print(f"Fixed Data Split: Train={len(train_df)}, Val={len(val_df)}, Test={len(test_df_with_contexts)}")

epochs = 50 # Maximum epochs for each trial
context_dim = sbert_model.get_sentence_embedding_dimension() # Get SBERT embedding dimension

# --- Hyperparameter Search Loop ---
for trial_num in range(num_trials):
    print(f"\n==================== Trial {trial_num + 1}/{num_trials} ====================")

    # Randomly select hyperparameters for the current trial
    current_params = {k: random.choice(v) for k, v in param_grid.items()}
    
    # Ensure embed_dim is not too small relative to context_dim or hidden_dims to avoid issues
    # This is a soft check; adjust ranges in param_grid for robust solutions
    if current_params['embed_dim'] * 2 + context_dim < current_params['hidden_dims'][0]:
        print(f"Warning: Initial MLP input dim ({current_params['embed_dim'] * 2 + context_dim}) is smaller than first hidden layer ({current_params['hidden_dims'][0]}). This might not be optimal. Skipping this configuration.")
        # Optionally skip this trial or adjust hidden_dims to be smaller
        continue
    
    print(f"Current Parameters: {current_params}")

    # Create DataLoaders for the current trial
    train_dataset = UCAMDataset(
        train_df['user_idx'].values,
        train_df['item_idx'].values,
        train_df['stars'].values.astype(np.float32),
        np.stack(train_df['context_vectors'].values) # Convert list of arrays to 2D array
    )
    val_dataset = UCAMDataset(
        val_df['user_idx'].values,
        val_df['item_idx'].values,
        val_df['stars'].values.astype(np.float32),
        np.stack(val_df['context_vectors'].values)
    )
    # 🚨 수정된 부분: test_df 대신 test_df_with_contexts 사용
    test_dataset = UCAMDataset(
        test_df_with_contexts['user_idx'].values,
        test_df_with_contexts['item_idx'].values,
        test_df_with_contexts['stars'].values.astype(np.float32),
        np.stack(test_df_with_contexts['context_vectors'].values)
    )

    train_loader = DataLoader(train_dataset, batch_size=current_params['batch_size'], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=current_params['batch_size'], shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=current_params['batch_size'], shuffle=False)

    # Model, Loss Function, and Optimizer are re-initialized for each trial
    model = UCAM(num_users=len(user2idx), num_items=len(item2idx),
                 context_dim=context_dim,
                 embed_dim=current_params['embed_dim'],
                 hidden_dims=current_params['hidden_dims']).to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=current_params['learning_rate'])
    criterion = nn.MSELoss()
    
    # Path to save the best model for this *specific trial*
    trial_model_save_path = f'temp_ucam_model_trial_{trial_num+1}.pt'

    trial_best_val_rmse = float('inf')
    epochs_no_improve = 0
    patience = current_params['patience'] # Use patience from current_params

    # --- Training Loop for current trial ---
    print("모델 학습 시작...")
    for epoch in range(epochs):
        model.train()
        total_train_loss = 0
        train_bar = tqdm(train_loader, desc=f"[Trial {trial_num+1}, Epoch {epoch+1}] Training", leave=False)

        for user_ids, item_ids, context_vectors, stars in train_bar:
            user_ids = user_ids.to(device)
            item_ids = item_ids.to(device)
            context_vectors = context_vectors.to(device)
            stars = stars.to(device)

            optimizer.zero_grad()
            predictions = model(user_ids, item_ids, context_vectors)
            loss = criterion(predictions, stars)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
            train_bar.set_postfix(loss=loss.item())
            
        # --- Validation after each epoch ---
        model.eval()
        val_mae, val_mse, val_rmse, val_mape = evaluate_model(model, val_loader, device)

        print(f"Epoch {epoch+1} | Train Loss: {total_train_loss / len(train_loader):.4f} | "
              f"Val RMSE: {val_rmse:.4f}, MAE: {val_mae:.4f}, MAPE: {val_mape:.2f}%")

        # Early stopping logic for this trial
        if val_rmse < trial_best_val_rmse - min_delta:
            trial_best_val_rmse = val_rmse
            epochs_no_improve = 0
            torch.save(model.state_dict(), trial_model_save_path)
            # print(f"    --> Improved. Model saved (RMSE: {trial_best_val_rmse:.4f})") # Optional: detailed logging
        else:
            epochs_no_improve += 1
            # print(f"    --> No improvement. ({epochs_no_improve}/{patience})")
            if epochs_no_improve == patience:
                print("Early stopping triggered for this trial.")
                break
    
    # --- Evaluate the best model from this trial on the Test Set ---
    if os.path.exists(trial_model_save_path):
        model.load_state_dict(torch.load(trial_model_save_path))
        print(f"\nLoaded best model for trial {trial_num+1} from {trial_model_save_path}")
        test_mae, test_mse, test_rmse, test_mape = evaluate_model(model, test_loader, device)
    else:
        print(f"\nWarning: Best model for trial {trial_num+1} not found. Testing with last model state.")
        test_mae, test_mse, test_rmse, test_mape = evaluate_model(model, test_loader, device)

    print(f"\n✅ Trial {trial_num+1} Test Results:")
    print(f" - MSE  : {test_mse:.4f}")
    print(f" - RMSE : {test_rmse:.4f}")
    print(f" - MAE  : {test_mae:.4f}")
    print(f" - MAPE : {test_mape:.2f}%")

    # Store results for this trial
    trial_results = {
        'trial_num': trial_num + 1,
        'parameters': current_params,
        'test_mse': test_mse,
        'test_rmse': test_rmse,
        'test_mae': test_mae,
        'test_mape': test_mape
    }
    results_log.append(trial_results)

    # Check if this trial yielded the overall best RMSE
    if test_rmse < best_rmse:
        best_rmse = test_rmse
        best_params = current_params
        # Save the overall best model
        torch.save(model.state_dict(), 'best_overall_ucam_model.pt')
        print(f"  --> New overall best RMSE found: {best_rmse:.4f} with params: {best_params}")

    # Clean up the temporary model file for this trial
    if os.path.exists(trial_model_save_path):
        os.remove(trial_model_save_path)

# -------------------- Final Results Output --------------------
print(f"\n--- Hyperparameter Search Completed ---")
print(f"Overall Best RMSE found: {best_rmse:.4f}")
print(f"Optimal Parameters: {best_params}")


Using device: cuda
✅ JSON 파일 로드 성공.
SBERT 모델 로딩 중...
SBERT 모델 로딩 완료.

--- Starting Hyperparameter Search with 10 trials ---
Data Split Random State: 42
SBERT 문맥 벡터 생성 중 (전체 데이터)...


Batches: 100%|██████████| 13994/13994 [06:23<00:00, 36.44it/s] 


Fixed Data Split: Train=313456, Val=44780, Test=89560

==================== Trial 1/10 ====================
Current Parameters: {'embed_dim': 128, 'hidden_dims': [256, 128], 'learning_rate': 0.001, 'batch_size': 128, 'patience': 10}
모델 학습 시작...


Epoch 1 | Train Loss: 0.8242 | Val RMSE: 0.8432, MAE: 0.6711, MAPE: 24.08%


Epoch 2 | Train Loss: 0.6458 | Val RMSE: 0.8237, MAE: 0.6498, MAPE: 23.92%


...

Epoch 17 | Train Loss: 0.4277 | Val RMSE: 0.7736, MAE: 0.5926, MAPE: 21.36%


Epoch 18 | Train Loss: 0.4174 | Val RMSE: 0.7747, MAE: 0.5937, MAPE: 21.12%


Epoch 19 | Train Loss: 0.4073 | Val RMSE: 0.7792, MAE: 0.5955, MAPE: 21.56%


Epoch 20 | Train Loss: 0.3979 | Val RMSE: 0.7789, MAE: 0.5966, MAPE: 21.33%
Early stopping triggered for this trial.

Loaded best model for trial 10 from temp_ucam_model_trial_10.pt

✅ Trial 10 Test Results:
 - MSE  : 0.5748
 - RMSE : 0.7582
 - MAE  : 0.5862
 - MAPE : 20.95%

--- Hyperparameter Search Completed ---
Overall Best RMSE found: 0.7545
Optimal Parameters: {'embed_dim': 32, 'hidden_dims': [64, 32], 'learning_rate': 0.0005, 'batch_size': 128, 'patience': 7}


TypeError: Object of type float32 is not JSON serializable

In [3]:
import json
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import sys

# -------------------- Utility Functions --------------------
def mean_absolute_percentage_error(y_true, y_pred):
    """
    Calculates MAPE, preventing division by zero.
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    epsilon = 1e-10
    return np.mean(np.abs((y_true - y_pred) / (y_true + epsilon))) * 100

# -------------------- Step 1: 데이터 로드 및 전처리 (JSON 파일용) --------------------
# 'review.json' 파일을 올바르게 로드합니다.
try:
    df = pd.read_json('review.json', lines=True)
    print("✅ JSON 파일 로드 성공.")
except FileNotFoundError:
    print("❌ Error: 'review.json' 파일이 존재하지 않습니다.")
    sys.exit()
except ValueError as e:
    print(f"JSONL 형식이 아닌 것 같습니다. 일반 JSON 파일로 다시 시도합니다. (오류: {e})")
    try:
        df = pd.read_json('review.json')
        print("✅ 일반 JSON 파일 로드 성공.")
    except Exception as e:
        print(f"❌ Error: 'review.json' 파일을 읽는 데 실패했습니다. 파일 형식을 확인해주세요. (오류: {e})")
        sys.exit()

df = df[['user_id', 'business_id', 'stars', 'text']].dropna()

# -------------------- Dataset 클래스 정의 --------------------
class UCAMDataset(Dataset):
    def __init__(self, users, items, ratings, contexts):
        self.users = users
        self.items = items
        self.ratings = ratings
        self.contexts = contexts

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        return (
            torch.tensor(self.users[idx], dtype=torch.long),
            torch.tensor(self.items[idx], dtype=torch.long),
            torch.tensor(self.contexts[idx], dtype=torch.float32),
            torch.tensor(self.ratings[idx], dtype=torch.float32)
        )

# -------------------- 모델 정의 (하이퍼파라미터 적용) --------------------
class UCAM(nn.Module):
    def __init__(self, num_users, num_items, context_dim=384, embed_dim=32):
        super().__init__()
        self.user_embed = nn.Embedding(num_users + 1, embed_dim)
        self.item_embed = nn.Embedding(num_items + 1, embed_dim)
        
        # 최적의 hidden_dims: [64, 32] 적용
        self.fc_layers = nn.Sequential(
            nn.Linear(embed_dim * 2 + context_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, user_ids, item_ids, context_vecs):
        u = self.user_embed(user_ids)
        i = self.item_embed(item_ids)
        x = torch.cat([u, i, context_vecs], dim=-1)
        return self.fc_layers(x).squeeze()

# -------------------- 실험을 위한 함수 정의 (하이퍼파라미터 적용) --------------------
def run_experiment(df, random_seed):
    print(f"\n==================== 실험 시작: random_state={random_seed} ====================")
    
    # 데이터 분할
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=random_seed)

    # 훈련 세트의 고유한 사용자/아이템 ID를 기반으로 인덱스를 생성합니다.
    user2idx = {uid: i for i, uid in enumerate(train_df['user_id'].unique())}
    item2idx = {iid: i for i, iid in enumerate(train_df['business_id'].unique())}

    # 테스트 세트에만 존재하는 사용자/아이템을 위한 '알 수 없음' 인덱스를 추가합니다.
    unknown_user_idx = len(user2idx)
    unknown_item_idx = len(item2idx)

    # SBERT 모델 로딩 및 문맥 벡터 생성
    print("SBERT 모델 로딩 및 문맥 벡터 생성 중...")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    sbert = SentenceTransformer('all-MiniLM-L6-v2', device=device)
    train_context_vectors = sbert.encode(train_df['text'].tolist(), show_progress_bar=True)
    test_context_vectors = sbert.encode(test_df['text'].tolist(), show_progress_bar=True)

    # 사용자/아이템 ID를 인덱스로 매핑하고, '알 수 없음'은 새로운 인덱스로 채웁니다.
    train_df['user'] = train_df['user_id'].map(user2idx)
    train_df['item'] = train_df['business_id'].map(item2idx)
    test_df['user'] = test_df['user_id'].map(user2idx).fillna(unknown_user_idx).astype(int)
    test_df['item'] = test_df['business_id'].map(item2idx).fillna(unknown_item_idx).astype(int)

    # DataLoader 생성 (최적의 batch_size: 128 적용)
    train_dataset = UCAMDataset(
        train_df['user'].values,
        train_df['item'].values,
        train_df['stars'].values.astype(np.float32),
        train_context_vectors
    )
    test_dataset = UCAMDataset(
        test_df['user'].values,
        test_df['item'].values,
        test_df['stars'].values.astype(np.float32),
        test_context_vectors
    )

    train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

    # 모델, 옵티마이저, 손실 함수 정의 (최적의 embed_dim: 32, learning_rate: 0.0005 적용)
    context_dim = sbert.get_sentence_embedding_dimension()
    model = UCAM(num_users=len(user2idx), num_items=len(item2idx), embed_dim=32, context_dim=context_dim).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
    criterion = nn.MSELoss()

    best_val_rmse = float('inf')
    epochs_no_improve = 0
    patience = 7  # 최적의 patience: 7 적용
    min_delta = 0.001
    epochs = 50
    model_path = f'best_ucam_model_{random_seed}.pt'

    print("모델 학습 시작...")
    for epoch in range(epochs):
        model.train()
        total_train_loss = 0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} (Train)", leave=False)

        for user_ids, item_ids, context_vectors, stars in progress_bar:
            user_ids = user_ids.to(device)
            item_ids = item_ids.to(device)
            context_vectors = context_vectors.to(device)
            stars = stars.to(device)

            optimizer.zero_grad()
            predictions = model(user_ids, item_ids, context_vectors)
            loss = criterion(predictions, stars)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
            
        # --- 검증 ---
        model.eval()
        val_preds, val_true = [], []
        with torch.no_grad():
            for user_ids, item_ids, context_vectors, stars in test_loader:
                user_ids = user_ids.to(device)
                item_ids = item_ids.to(device)
                context_vectors = context_vectors.to(device)
                stars = stars.to(device)

                preds = model(user_ids, item_ids, context_vectors)
                val_preds.extend(preds.cpu().numpy())
                val_true.extend(stars.cpu().numpy())

        val_preds = np.array(val_preds)
        val_true = np.array(val_true)
        val_mse = mean_squared_error(val_true, val_preds)
        val_rmse = np.sqrt(val_mse)
        val_mae = mean_absolute_error(val_true, val_preds)
        val_mape = mean_absolute_percentage_error(val_true, val_preds)

        print(f"\nEpoch {epoch+1} | Train Loss: {total_train_loss / len(train_loader):.4f} | "
              f"Val RMSE: {val_rmse:.4f}, MAE: {val_mae:.4f}, MAPE: {val_mape:.2f}%")

        if val_rmse < best_val_rmse - min_delta:
            best_val_rmse = val_rmse
            epochs_no_improve = 0
            torch.save(model.state_dict(), model_path)
            print(f"  --> 개선됨. 모델 저장됨 (RMSE: {best_val_rmse:.4f})")
        else:
            epochs_no_improve += 1
            print(f"  --> 개선 없음. ({epochs_no_improve}/{patience})")
            if epochs_no_improve == patience:
                print("조기 종료 발생.")
                break
    
    # -------------------- 테스트셋 평가 --------------------
    def evaluate_model(model, data_loader, device):
        model.eval()
        preds, targets = [], []

        with torch.no_grad():
            for users, items, contexts, ratings in data_loader:
                users = users.to(device)
                items = items.to(device)
                contexts = contexts.to(device)
                ratings = ratings.to(device)

                output = model(users, items, contexts)
                preds.extend(output.cpu().numpy())
                targets.extend(ratings.cpu().numpy())

        preds = np.array(preds)
        targets = np.array(targets)

        mae = mean_absolute_error(targets, preds)
        mse = mean_squared_error(targets, preds)
        rmse = np.sqrt(mse)
        mape = mean_absolute_percentage_error(targets, preds)

        return mae, mse, rmse, mape

    if os.path.exists(model_path):
        model.load_state_dict(torch.load(model_path))
        print(f"\n최적 모델 로드 완료: {model_path}")
        mae, mse, rmse, mape = evaluate_model(model, test_loader, device)

        print(f"\n✅ [UCAM] 최종 테스트 평가 지표 (random_state={random_seed}):")
        print(f"    - MSE  : {mse:.4f}")
        print(f"    - RMSE : {rmse:.4f}")
        print(f"    - MAE  : {mae:.4f}")
        print(f"    - MAPE : {mape:.2f}%")
        
        return {'mse': mse, 'rmse': rmse, 'mae': mae, 'mape': mape}
    
    return None

# -------------------- Step 9: 5회 실험 및 평균 계산 --------------------
all_results = []
num_runs = 5
start_seed = 42

for i in range(num_runs):
    seed = start_seed + i
    results = run_experiment(df, seed)
    if results:
        all_results.append(results)

if all_results:
    avg_mse = np.mean([r['mse'] for r in all_results])
    avg_rmse = np.mean([r['rmse'] for r in all_results])
    avg_mae = np.mean([r['mae'] for r in all_results])
    avg_mape = np.mean([r['mape'] for r in all_results])

    print("\n\n==================== 5회 실험 평균 결과 ====================")
    print(f"✔️ 평균 MSE  : {avg_mse:.4f}")
    print(f"✔️ 평균 RMSE : {avg_rmse:.4f}")
    print(f"✔️ 평균 MAE  : {avg_mae:.4f}")
    print(f"✔️ 평균 MAPE : {avg_mape:.2f}%")
else:
    print("❌ 실험 결과를 얻지 못했습니다. 오류를 확인해주세요.")

✅ JSON 파일 로드 성공.

==================== 실험 시작: random_state=42 ====================
SBERT 모델 로딩 및 문맥 벡터 생성 중...


Batches: 100%|██████████| 2799/2799 [00:57<00:00, 49.01it/s] 


모델 학습 시작...



Epoch 1 | Train Loss: 1.0736 | Val RMSE: 0.8306, MAE: 0.6574, MAPE: 24.22%
  --> 개선됨. 모델 저장됨 (RMSE: 0.8306)



Epoch 2 | Train Loss: 0.6514 | Val RMSE: 0.7920, MAE: 0.6199, MAPE: 22.38%
  --> 개선됨. 모델 저장됨 (RMSE: 0.7920)


...


Epoch 13 | Train Loss: 0.3996 | Val RMSE: 0.7617, MAE: 0.5888, MAPE: 20.62%
  --> 개선 없음. (3/7)



Epoch 14 | Train Loss: 0.3871 | Val RMSE: 0.7624, MAE: 0.5836, MAPE: 21.02%
  --> 개선 없음. (4/7)



Epoch 15 | Train Loss: 0.3749 | Val RMSE: 0.7694, MAE: 0.5933, MAPE: 20.59%
  --> 개선 없음. (5/7)



Epoch 16 | Train Loss: 0.3627 | Val RMSE: 0.7695, MAE: 0.5896, MAPE: 20.91%
  --> 개선 없음. (6/7)



Epoch 17 | Train Loss: 0.3510 | Val RMSE: 0.7740, MAE: 0.5931, MAPE: 20.97%
  --> 개선 없음. (7/7)
조기 종료 발생.

최적 모델 로드 완료: best_ucam_model_46.pt

✅ [UCAM] 최종 테스트 평가 지표 (random_state=46):
    - MSE  : 0.5594
    - RMSE : 0.7479
    - MAE  : 0.5750
    - MAPE : 20.55%


==================== 5회 실험 평균 결과 ====================
✔️ 평균 MSE  : 0.5648
✔️ 평균 RMSE : 0.7515
✔️ 평균 MAE  : 0.5795
✔️ 평균 MAPE : 20.61%
